<img src="resources/cifar10.png" width="700" height="700" />

*<p style="text-align: center;">Figure 1: Categories and Color Images Example from CIFAR-10 Dataset</p>*

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms


USERNAME = 'adi' # Change this to your username.

DOWNLOAD_FOLDER = '/home/idies/workspace/Temporary/' + USERNAME + '/scratch'

BATCH_SIZE = 10
LEARNING_RATE = 0.001
ITERATIONS = 5

print_frequency = 500

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root=DOWNLOAD_FOLDER, train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root=DOWNLOAD_FOLDER, train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import torch
import torch.nn as nn # This is the Neural Networks package. We use this to create our Machine Learning Models.

class SimpleConvolutionalNeuralNetworkModel(nn.Module):
    def __init__(self):
        super(SimpleConvolutionalNeuralNetworkModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, input_data):
        x = self.pool(nn.functional.relu(self.conv1(input_data)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        output = self.fc3(x)

        return output

<img src="resources/cnn example.png" width="700" height="700" />

*<p style="text-align: center;">Figure 2: Example of a Convolutional Neural Network</p>*

Convolution part is extracting info

In [ ]:
import torch.optim as optim
import time

model = None
model = SimpleConvolutionalNeuralNetworkModel()


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

iters = 0
elapsed_time = 0
start = time.time()
for epoch in range(ITERATIONS):  # loop over the dataset multiple times 
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        iters += 1
        if iters % print_frequency == 0:    # print every 500 iterations.
            correct = 0
            total = 0
            with torch.no_grad():
                for data in testloader:
                    images, labels = data
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            accuracy = 100 * correct/total
            elapsed_time += time.time() - start
            print('Elapsed Time: %.2f sec, Iteration: %d, Loss: %f, Accuracy: %.2f' % (elapsed_time, iters, running_loss/print_frequency, accuracy))
            running_loss = 0
    print('')
print('Finished Training')

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))